In [1]:
import torch
import torch.nn as nn 
import torchvision
import torchvision.transforms as transforms

import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import numpy as np

In [1]:
import torch
import torch.nn as nn 
import torchvision
import torchvision.transforms as transforms

# device config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')


max_epoch = 15
layers = 6

class Serf(nn.Module):
        def __init__(self):
            super(Serf, self).__init__()
        def forward(self, X):
            return torch.erf(torch.nn.functional.softplus(X)) * X

def createAct(activation="relu"):  
    if activation == "silu":
      return nn.SiLU()
    elif activation == "relu":
      return nn.ReLU()
    elif activation == "mish":
      return torch.nn.Mish()
    elif activation == "serf":
      return Serf()


class ANN(nn.Module):
  def __init__(self, n_layers, n_input, n_hidden, n_class, activation):
    super(ANN, self).__init__()
    self.first = nn.Linear(n_input, n_hidden)
    self.firstbn = nn.BatchNorm1d(num_features=n_hidden) 
    self.firstdp = nn.Dropout(p=0.25)

    self.n_layers = n_layers
    self.n_class = n_class
    self.n_hidden = n_hidden
    self.activation = activation

    self.l = nn.ModuleList()  

    self.bn = nn.ModuleList()

    self.dp = nn.ModuleList()

    for i in range (self.n_layers):
      self.l.append(nn.Linear(self.n_hidden, n_hidden))
      self.bn.append(nn.BatchNorm1d(num_features=n_hidden))
      self.dp.append(nn.Dropout(p=0.25))
    self.final = nn.Linear(n_hidden, n_class) # 10 classes

  def forward(self, X):
    out = self.firstbn(self.firstdp(createAct(self.activation)(self.first(X))))
    for i in range (self.n_layers):      
      out = self.bn[i](self.dp[i](createAct(self.activation)(self.l[i](out))))
    return self.final(out)

network = ANN(layers, 2, 512, 1, "relu").to(device)

network.eval()
X = torch.tensor([[1000,1000]], dtype=torch.float);
print(network(X))

In [3]:
import matplotlib.pyplot as plt
import numpy as np

IMG_SIZE = 100

arr = np.empty(shape=(IMG_SIZE,IMG_SIZE))
for i in range(IMG_SIZE):
    for j in range(IMG_SIZE):
        X = torch.tensor([[i, j]], dtype=torch.float).cuda();
        arr[i][j] = float(network(X)[0])
            
plt.imshow(arr)

In [8]:
plt.imsave("relu_landscape.png", arr)